In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/mypythondirectory')

In [3]:
import os
os.chdir("/content/drive/MyDrive/MTech_Project_Code/")
!ls

00000_00000.jpg      GTSRB		pr_curves.ipynb
00000_00001.jpg      labels.csv		pr_curves.py
12.jpg		     lib		__pycache__
14.jpg		     model		runs
2.jpg		     myData		summarize_model.ipynb
31.jpg		     my_model_ores.pt	summarize_model.py
36.jpg		     my_model_ores.pth	train.ipynb
build_dataset.ipynb  my_model_pres.pt	train.py
build_dataset.py     my_model_pres.pth	Untitled.ipynb
config.py	     my_model.pt	utils
data		     my_model.pth	visualize_model.ipynb
evaluate.ipynb	     my_modelres.pt	visualize_model.py
evaluate.py	     my_modelres.pth


In [4]:
!nvidia-smi

Sat Apr 16 01:05:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from torch import nn, optim
import torch

In [6]:
from evaluate import evaluate
from model.data_loader import train_data_loader, val_data_loader
from model.net import loss_fn, net
from utils.device import device
from utils.tensorboard import plot_classes_preds
from utils.writer import writer
import config

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
def train(
        net,
        optimizer,
        scheduler,
        loss_fn,
        train_data_loader,
        val_data_loader,
        model_path,
        num_epochs,
        device,
        writer,
        model_path1):
    max_val_accuracy = 0.0

    # Loop over the dataset multiple times
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('----------')
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = net(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # Log statistics
            running_loss += loss.item()
            steps = 50
            if i % steps == steps - 1:
                # Log the running loss
                # print(f'- {i + 1} mini batches\tloss {round(running_loss / steps, 3)}')
                writer.add_scalar('training loss',
                                  running_loss / steps,
                                  epoch * len(train_data_loader) + i)
                # Log a Matplotlib Figure showing the model's predictions on a random mini-batch
                writer.add_figure('predictions vs. actuals',
                                  plot_classes_preds(net, inputs, labels),
                                  global_step=epoch * len(train_data_loader) + i)
                running_loss = 0.0
        train_accuracy, train_loss, train_num_inputs = evaluate(net, loss_fn, train_data_loader, device)
        val_accuracy, val_loss, val_num_inputs = evaluate(net, loss_fn, val_data_loader, device)
        scheduler.step(val_loss)

        # Log statistics
        print(f'Train\taccuracy {round(train_accuracy * 100, 2)}%\tloss {round(train_loss, 3)}\timages {train_num_inputs}')
        print(f'Val\taccuracy {round(val_accuracy * 100, 2)}%\tloss {round(val_loss, 3)}\timages {val_num_inputs}')
        print()
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        writer.add_scalar('Accuracy/val', val_accuracy, epoch)

        # Save model
        if val_accuracy > max_val_accuracy:
            max_val_accuracy = val_accuracy
            torch.save(net.state_dict(), model_path)
            torch.save(net.state_dict(), model_path1)
            print(f'Found better model. Saved at {model_path}.')

In [8]:
if __name__ == '__main__':
    learning_rate = 0.001
    num_epochs = 20

    net.to(device)
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    train(
        net,
        optimizer,
        scheduler,
        loss_fn,
        train_data_loader,
        val_data_loader,
        config.model_path,
        num_epochs,
        device,
        writer,
        config.model_path1)

    writer.close()


Epoch 1/20
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train	accuracy 67.9%	loss 1.03	images 31387
Val	accuracy 70.44%	loss 0.947	images 7842

Found better model. Saved at /content/drive/MyDrive/MTech_Project_Code/my_model_ores.pth.
Epoch 2/20
----------
Train	accuracy 32.21%	loss 2.261	images 31387
Val	accuracy 33.51%	loss 2.201	images 7842

Epoch 3/20
----------
Train	accuracy 60.13%	loss 1.208	images 31387
Val	accuracy 61.69%	loss 1.141	images 7842

Epoch 4/20
----------
Train	accuracy 75.74%	loss 0.7	images 31387
Val	accuracy 79.6%	loss 0.584	images 7842

Found better model. Saved at /content/drive/MyDrive/MTech_Project_Code/my_model_ores.pth.
Epoch 5/20
----------
Train	accuracy 84.41%	loss 0.45	images 31387
Val	accuracy 88.64%	loss 0.339	images 7842

Found better model. Saved at /content/drive/MyDrive/MTech_Project_Code/my_model_ores.pth.
Epoch 6/20
----------
Train	accuracy 90.18%	loss 0.288	images 31387
Val	accuracy 93.53%	loss 0.194	images 7842

Found better model. Saved at /content/drive/MyDrive/MTech_Project_Code/my_model_ores.p